#### Data from [Yelp Dataset JSON](https://www.yelp.com/dataset)

In [1]:
# Import dependencies
import pandas as pd
import json

# Import NLTK
import nltk
from nltk.tokenize import word_tokenize as wt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer as lmtzr 


In [2]:
# Create and Open data file
data_file = open('../../Data/archive/yelp_academic_dataset_review.json', encoding = 'utf8')
data = []

# Read in data
for line in data_file:
    data.append(json.loads(line))

review_df = pd.DataFrame(data)
data_file.close()

In [26]:
# review_df.head()
review_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [5]:
# Set stop words
stop_words = set(stopwords.words('english'))

In [12]:
# Drop all positive reviews and reset index
bad_review_df = review_df[review_df['stars'] < 3.0].reset_index()

In [28]:
test = wt(bad_review_df['text'][5])

In [30]:
new_text = nltk.Text(test)

In [40]:
new_text.concordance('artichoke')

Displaying 1 of 1 matches:
We started with the artichoke dip , big mistake . It was cold an


In [36]:
bad_review_df['text'][5]

'We started with the artichoke dip, big mistake. It was cold and the chips were stale. Mentioned to our waitress, but nothing came of it.  We ordered a large pizza, thin crust which after adding toppings it was a bit pricey for pizza.  Pizza was ok, not great. Except for not being our advocate our server was good.  I have been here before for Cardinals game, it was better then. Will try another venue next time.'